In [1]:
import csv
import pandas as pd

In [3]:
data = pd.read_csv("C:/Users/HP/Documents/work/DeepForm/deepform-master/source/ftf-all-filings.tsv", delimiter = '\t')

In [4]:
data.head()

,id,filing_type,contract_number,url,committee,agency,callsign,dc_slug,thumbnail_url,gross_amount,market_id,upload_date
0,23969,Federal,314271,/collect/files/59438/Political File/2012/Feder...,NRSC-MANDEL/REPUBLICAN,STRATEGIC MEDIA SERVICES,WCPO-TV,459790-nrsc_mandel_rep_senate_1010_101412_314271,https://s3.amazonaws.com/s3.documentcloud.org/...,21100.0,35.0,2012-10-10 00:00:00 UTC
1,2333,Non-Candidate Issue Ads,9792042,/collect/files/11289/Political File/2012/Non-C...,60+ ASSOCIATION,MENTZER MEDIA SERVICES,WKRC-TV,416491-collect-files-11289-political-file-2012...,https://s3.amazonaws.com/s3.documentcloud.org/...,6250.0,35.0,2012-08-14 00:00:00 UTC
2,31990,Federal,336298,/collect/files/73910/Political File/2012/Feder...,POL/R TERRY/D/PRES/US,KATHLEEN OFFERMAN,WPXI,470297-101612-terry-contract-2-13504017232906-...,https://s3.amazonaws.com/s3.documentcloud.org/...,30.0,23.0,2012-10-16 00:00:00 UTC
3,33211,Federal,116252,/collect/files/73123/Political File/2012/Feder...,COMMITTEE TO ELECT SHANELLE JACKSON,SHANELLE JACKSON FOR CONGRESS,WJBK,473630-116252-0-13442821773323-_-pdf,https://s3.amazonaws.com/s3.documentcloud.org/...,1170.0,11.0,2012-08-06 00:00:00 UTC
4,6213,Non-Candidate Issue Ads,48982,/collect/files/72313/Political File/2012/Non-C...,PRIORITIES USA ACTION,MUNDY KATOWITZ MEDIA,WHP-TV,420426-collect-files-72313-political-file-2012...,https://s3.amazonaws.com/s3.documentcloud.org/...,9335.0,41.0,2012-08-13 00:00:00 UTC


In [11]:
data['dc_slug'][2]

'470297-101612-terry-contract-2-13504017232906-_-pdf'

In [16]:
#for index, row in data.iterrows(0:3):
 #   print(data['dc_slug'])
    
for i in range(3) :
    slug = data.loc[i, "dc_slug"]
    print(slug)

459790-nrsc_mandel_rep_senate_1010_101412_314271
416491-collect-files-11289-political-file-2012-non
470297-101612-terry-contract-2-13504017232906-_-pdf


In [17]:
import wandb

In [18]:
import wandb.keras as wk

In [22]:
# Data extraction by deep learning, using a fully connected architecture over token windows.
# Engineered to extract total amounts, using a few custom features.
# Achieves up to 90% accuracy.
#
# jstray 2019-6-12

import keras as K
from keras.engine.input_layer import Input
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Lambda, concatenate
from keras.layers.embeddings import Embedding
from keras.backend import expand_dims, squeeze
import tensorflow as tf
import pandas as pd
import numpy as np
import csv
import re
import random
import pdfplumber
import os
import pickle
from decimal import Decimal

#import wandb
#from wandb.keras import WandbCallback

# uration
#run = init(project="jonathan_summer_1", entity="deepform", name="testing")

# = run.

In [23]:
# address some inteeface discrepancies when using tensorflow.keras
if "slice" not in K.__dict__ and K.backend() == "tensorflow":
    # this is a good indicator that we are using tensorflow.keras

    try:
        # at first try to monkey patch what we need, will only work if keras-team keras is installed
        from keras import backend as KKK

        try:
            K.__dict__.update(
                is_tensor=KKK.is_tensor,
                slice=KKK.slice,
            )
        finally:
            del KKK
    except ImportError:
        # if that doesn't work we do a dirty copy of the code required
        import tensorflow as tf
        from tensorflow.python.framework import ops as tf_ops


        def is_tensor(x):
            return isinstance(x, tf_ops._TensorLike) or tf_ops.is_dense_tensor_like(x)


        def slice(x, start, size):
            x_shape = K.int_shape(x)
            if (x_shape is not None) and (x_shape[0] is not None):
                len_start = K.int_shape(start)[0] if is_tensor(start) else len(start)
                len_size = K.int_shape(size)[0] if is_tensor(size) else len(size)
                if not (len(K.int_shape(x)) == len_start == len_size):
                    raise ValueError('The dimension and the size of indices should match.')
            return tf.slice(x, start, size)

TypeError: 'module' object is not callable

In [25]:
read_docs = 150 # how many docs to load, at most
window_len = 30 # size of token sequences to train on (and network size!)
vocab_size = 500
token_dims = 7 # number of features per token, including token hash
positive_fraction = 0.5
target_thresh = 0.9 # target match scores larger than this will becomes positive labels
epochs = 50
batch_size=10000
steps_per_epoch = 10
doc_acc_sample_size = 25 # how many documents to check extraction on after each epoch
penalize_missed = 5 # how much more a missed 1 counts than a missed 0 in output
val_split = 0.2
len_train = 80

source_data = 'C:/Users/HP/Documents/work/DeepForm/deepform-master/source/training.csv'
pickle_destination = 'C:/Users/HP/Documents/work/DeepForm/deepform-master/source/cached_features.p'

# ---- Load data and generate features ----

# Generator that reads raw training data
# For each document, yields an array of dictionaries, each of which is a token
def input_docs(max_docs=None):
	incsv = csv.DictReader(open(source_data, mode='r'))
		
	# Reconstruct documents by concatenating all rows with the same slug
	active_slug = None
	doc_rows = [] 
	num_docs = 0

	for row in incsv:	
		# throw out tokens that are too short, they won't help us
		token = row['token']
		if len(token) < 3:
			continue 

		if row['slug'] != active_slug:
			if active_slug:
				yield doc_rows
				num_docs += 1
				if max_docs and num_docs >= max_docs:
					return
			doc_rows = [row]
			active_slug = row['slug']
		else:
			doc_rows.append(row)
		
	yield doc_rows


def is_dollar_amount(s):
	return re.search(r'\$?\d[\d,]+(\.\d\d)?',s) != None

def token_features(row, vocab_size):
	tokstr = row['token'].upper()
	return [ hash(tokstr) % vocab_size,
					 float(row['page']), 
					 float(row['x0']),
					 float(row['y0']), 
					 float(len(tokstr)),
					 float(np.mean([c.isdigit() for c in tokstr])),
					 float(is_dollar_amount(tokstr)) ]
	
# Load raw training data, create our per-token features and binary labels	
def load_training_data_nocache():
	slugs = []
	token_text = []
	features = []
	labels = []
	for doc_tokens in input_docs(max_docs=read_docs):	
		if len(doc_tokens) < window_len:
			continue # TODO pad shorter docs
			
		# Not training data, but used for evaluating results later
		slugs.append(doc_tokens[0]['slug']) # unique document ID, also PDF filename
		token_text.append([row['token'] for row in doc_tokens])

		features.append([token_features(row, vocab_size) for row in doc_tokens])
		
		# threshold fuzzy matching score with our target field, to get binary labels 
		labels.append([(0 if float(row['gross_amount']) < target_thresh else 1) for row in doc_tokens])
	print("Length of slugs in load_training_data_nocache = ", len(slugs))
	return slugs, token_text, features, labels
	
# Because generating the list of features is so expensive, we cache it on disk
def load_training_data():
	if os.path.isfile(pickle_destination):
		print('Loading training data from cache...')
		slugs, token_text, features, labels = pickle.load(open(pickle_destination, 'rb'))
	else:
		print('Loading training data...')
		slugs, token_text, features, labels = load_training_data_nocache()
		print('Saving training data to cache...')
		pickle.dump((slugs, token_text, features, labels), open(pickle_destination, 'wb'))

	# Trim the training data so we can sweep across various training data sizes
	print("Length of slugs in load_training_data before modification = ", len(slugs))
	slugs = slugs[:len_train]
	print("Length of slugs in load_training_data after modification = ", len(slugs))
	token_text = token_text[:len_train]
	features = features[:len_train]
	labels = labels[:len_train]

	return slugs, token_text, features, labels

# ---- Resample features,labels as windows ----

# returns a window of tokens, labels at a random position in a random document
def one_window_unbalanced(features, labels, window_len):
	doc_idx = random.randint(0,len(features)-1)
	doc_len = len(features[doc_idx])
	tok_idx = random.randint(0, doc_len-window_len)
	return features[doc_idx][tok_idx : tok_idx+window_len], labels[doc_idx][tok_idx : tok_idx+window_len]

# control the fraction of windows that include a positive label. not efficient.
def one_window(features, labels, window_len, positive_fraction):
	f,l = one_window_unbalanced(features, labels, window_len)
	if random.random() > positive_fraction: # mostly positive examples
		while not 1 in l:
			f,l = one_window_unbalanced(features, labels, window_len)
	return f,l

def windowed_generator(features, labels, ):
	# Create empty arrays to contain batch of features and labels#
	batch_features = np.zeros((batch_size, window_len, token_dims))
	batch_labels = np.zeros((batch_size, window_len))

	while True:
		for i in range(batch_size):
			features1,labels1 = one_window(features, labels, window_len, positive_fraction)
			batch_features[i,:,:] = features1
			batch_labels[i,:] = labels1
		yield batch_features, batch_labels

# ---- Custom loss function is basically MSE but high penalty for missing a 1 label ---

def missed_token_loss(one_penalty):
   
	def _missed_token_loss(y_true, y_pred):
	  expected_zero = tf.cast(tf.math.equal(y_true,0), tf.float32)
	  s = y_pred*expected_zero
	  zero_loss = K.backend.mean(K.backend.square(s))
	  expected_one = tf.cast(tf.math.equal(y_true,1), tf.float32)
	  t = one_penalty*(1-y_pred)*expected_one
	  one_loss = K.backend.mean(K.backend.square(t))
	  return zero_loss + one_loss

	return _missed_token_loss # closes over one_penalty

# --- Specify network ---

def create_model():
	indata = Input((window_len, token_dims))

	# split into the hash and the rest of the token features, embed hash as one-hot, then merge
	tok_hash = Lambda( lambda x: squeeze(tf.slice(x, (0,0,0), (-1,-1,1)),axis=2))(indata)
	tok_features = Lambda( lambda x: tf.slice(x, (0,0,1), (-1,-1,-1)))(indata)
	embed = Embedding(vocab_size, 32)(tok_hash)
	merged = concatenate([embed, tok_features], axis=2)

	f = Flatten()(merged)
	d1 = Dense(window_len*token_dims*5, activation='sigmoid')(f)
	d2 = Dropout(0.3)(d1)
	d3 = Dense(window_len*token_dims, activation='sigmoid')(d2)
	d4 = Dropout(0.3)(d3)
	d5 = Dense(window_len, activation='elu')(d4)

	model = Model(inputs=[indata], outputs=[d5])
	model.compile(
		optimizer='adam', 
		loss=missed_token_loss(penalize_missed), 
		metrics=['acc'])

	return model

# --- Predict ---
# Our network is windowed, so we have to aggregate windows to get a final score

# Returns vector of token scores
def predict_scores(model, features, window_len):
	doc_len = len(features)
	num_windows = doc_len-window_len

	windowed_features = np.array([features[i:i+window_len] for i in range(num_windows)])
	window_scores = model.predict(windowed_features)

	scores = np.zeros(doc_len)
	for i in range(num_windows):
		scores[i:i+window_len] += window_scores[i] # would max work better than sum?
	return scores

# returns text, score of best answer
def predict_answer(model, features, token_text, window_len):
	scores = predict_scores(model, features, window_len)
	best_score_idx = np.argmax(scores)
	best_score_text = token_text[best_score_idx]
	return best_score_text, scores[best_score_idx]

# returns text of correct answer,
def correct_answer(features, labels, token_text):
	answer_idx = np.argmax(labels)
	answer_text = token_text[answer_idx]
	return answer_text

# Calculate accuracy of answer extraction over num_to_test docs, print diagnostics while we do so
def compute_accuracy(model, window_len, slugs, token_text, features, labels, num_to_test):
	acc = 0.0
	for i in range(num_to_test):
		doc_idx = random.randint(0, len(slugs)-1)
		predict_text, predict_score = predict_answer(model, features[doc_idx], token_text[doc_idx], window_len)
		answer_text = correct_answer(features[doc_idx], labels[doc_idx], token_text[doc_idx])
		print(f'{slugs[doc_idx]}: guessed "{predict_text}" with score {predict_score}, correct "{answer_text}"')
		if predict_text==answer_text:
			acc+=1
	return acc/num_to_test


# ---- Custom callback to log document-level accuracy ----

class DocAccCallback(K.callbacks.Callback):
	def __init__(self, window_len, slugs, token_text, features, labels, num_to_test, logname):
		self.window_len = window_len
		self.slugs = slugs
		self.token_text = token_text
		self.features = features
		self.labels = labels
		self.num_to_test = num_to_test
		self.logname = logname

	def on_epoch_end(self, epoch, logs):
		acc = compute_accuracy(self.model,
			self.window_len,
			self.slugs,
			self.token_text,
			self.features,
			self.labels,
			self.num_to_test+epoch)  # test more docs later in training, for more precise acc
		print(f'This epoch {self.logname}: {acc}')
		#wandb.log({self.logname:acc})


# --- MAIN ----

#print('uration:')
#print()

slugs, token_text, features, labels = load_training_data()
print(f'Loaded {len(features)}')
max_length = max([len(x) for x in labels])
print(f'Max document size {max_length}')
avg_length = sum([len(x) for x in labels])/len(labels)
print(f'Average document size {avg_length}')

# split into train and test
slugs_train = []
token_text_train = []
features_train = []
labels_train = []
slugs_val = []
token_text_val = []
features_val = []
labels_val = []
for i in range(len(features)):
	if random.random() < val_split:
		slugs_val.append(slugs[i])
		token_text_val.append(token_text[i])
		features_val.append(features[i])
		labels_val.append(labels[i])
	else:
		slugs_train.append(slugs[i])
		token_text_train.append(token_text[i])		
		features_train.append(features[i])
		labels_train.append(labels[i])

print(f'Training on {len(features_train)}, validating on {len(features_val)}')

model = create_model()
print(model.summary())


model.fit_generator(
	windowed_generator(features_train, labels_train, ),
	steps_per_epoch=steps_per_epoch,
	epochs=epochs,
	callbacks=[ 
		#WandbCallback(),
		DocAccCallback(	window_len, 
										slugs_train, 
										token_text_train, 
										features_train, 
										labels_train, 
										doc_acc_sample_size,
										'doc_train_acc'),
		DocAccCallback(	window_len, 
										slugs_val, 
										token_text_val, 
										features_val, 
										labels_val, 
										doc_acc_sample_size,
										'doc_val_acc')
	])

Loading training data from cache...
Length of slugs in load_training_data before modification =  100
Length of slugs in load_training_data after modification =  80
Loaded 80
Max document size 9873
Average document size 1692.5875
Training on 61, validating on 19
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 7)]      0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 30)           0           input_3[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 32)       16000       lambda_3[0][0]                   
______________________________

C:\Users\HP\anaconda3\Lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
Cause: code mixing tabs and spaces for indentation is not allowed
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: code mixing tabs and spaces for indentation is not allowed
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
10/10 [==============================] - 25s 919ms/step - loss: 1.0001 - acc: 0.0385
468469-menendez-order-4-13500774152684-_-pdf: guessed "$41,450.00" with score 13.516670793294907, correct "$41,450.00"
511281-336563-4-13524183201766-_-pdf: guessed "$22,780.00" with score 13.853281378746033, correct "$26,800.00"
470486-kirkpatrick_1009-1015rev6-13503492049654-_-pdf: guessed "KTVK" with score 13.831149652600288, correct "$20,030.00"
511316-obama-2051-final_341803-1-13524183386707-_-pdf: guessed "$162,190.00" with score 13.966634452342987, correct "$162,190.00"
435451-collect-files-23079-political-file-2012-non: guessed "$63,850.00" with score 14.089594632387161

511316-obama-2051-final_341803-1-13524183386707-_-pdf: guessed "$137,861.50" with score 13.216709703207016, correct "$162,190.00"
511318-obama-2175-final_343497-1-13524183408511-_-pdf: guessed "$6,000.00" with score 12.815026491880417, correct "$180,260.00"
470651-obama187040-13503351151533-_-pdf: guessed "$19,250.00" with score 13.665321260690689, correct "$19,250.00"
511318-obama-2175-final_343497-1-13524183408511-_-pdf: guessed "$6,000.00" with score 12.815026491880417, correct "$180,260.00"
511319-rncromney-483-final_339838-1-13524180984389-_-pdf: guessed "$1,620.00" with score 13.297474771738052, correct "$226,220.00"
442594-collect-files-73195-political-file-2012-non: guessed "$6,350.00" with score 13.643945574760437, correct "$6,350.00"
420426-collect-files-72313-political-file-2012-non: guessed "$9,335.00" with score 13.666707247495651, correct "$9,335.00"
419761-collect-files-65684-political-file-2012-federal: guessed "$120,375.00" with score 13.340027391910553, correct "$120,

KeyboardInterrupt: 